In [183]:
import enum

class Player(enum.Enum):
    black = 1  # Attackers: goes first
    white = 2  # Defenders

    @property
    def other(self):
        return Player.black if self == Player.white else Player.white


In [184]:
p1 = Player.black
p2 = Player.white

assert p1.other == Player.white
print(p1.value)

1


In [185]:
from collections import namedtuple

class Point(namedtuple("Point", "row col")):
    def neighbors(self):
        return [
            Point(self.row - 1, self.col),  # Up
            Point(self.row + 1, self.col),  # Down
            Point(self.row, self.col - 1),  # Left
            Point(self.row, self.col + 1),  # Right
        ]

    def vert_neighbors(self):
        return [
            Point(self.row - 1, self.col),  # Up
            Point(self.row + 1, self.col),  # Down
        ]

    def hor_neighbors(self):
        return [
            Point(self.row, self.col - 1),  # Left
            Point(self.row, self.col + 1),  # Right
        ]

    def __deepcopy__(self, memodict={}):
        return self


In [186]:
point = Point(1, 2)

point.neighbors()[0].col

2

In [236]:
PAWN_TO_CHAR = {
    None: '.',
    Player.black: 'B',
    Player.white: 'W',
}

def print_move(board, point):
    char = PAWN_TO_CHAR.get(board[point.row, point.col], '.')
    print(f"Move to {point} with pawn {char}")

def print_board(board):
    for row in board:
        print(' '.join(PAWN_TO_CHAR.get(cell, '.') for cell in row))
    print()

def point_from_coords(row, col):
    return Point(row, col)

def coords_from_point(point):
    return point.row, point.col

def decode_action(move):
    from_row = move // (11*4*10)
    rem = move % (11*4*10)

    from_col = rem // (4*10)
    rem = rem % (4*10)

    direction = rem // 10
    distance = (rem % 10) + 1

    return from_row, from_col, direction, distance

def encode_action(from_row, from_col, direction, distance):
    if not (0 <= from_row < 11 and 0 <= from_col < 11):
        raise ValueError("Row and column must be between 0 and 10 inclusive")
    if direction not in [0, 1, 2, 3]:
        raise ValueError("Direction must be one of: 0 (up), 1 (down), 2 (left), 3 (right)")
    if not (0 <= distance <= 10):
        raise ValueError("Distance must be between 0 and 10 inclusive")

    return from_row * (11 * 4 * 10) + from_col * (4 * 10) + direction * 10 + (distance - 1)


def calculate_end_position(from_col, from_row, direction, distance):
    if direction == 0:  # Up
        return Point(from_row - distance, from_col)
    elif direction == 1:  # Down
        return Point(from_row + distance, from_col)
    elif direction == 2:  # Left
        return Point(from_row, from_col - distance)
    elif direction == 3:  # Right
        return Point(from_row, from_col + distance)
    else:
        raise ValueError("Invalid direction")


class Move:
    def __init__(self, from_pos, to_pos):
        self.from_pos = from_pos
        self.to_pos = to_pos

    @classmethod
    def from_encoded(cls, encoded_move, board_size=11):
        from_row, from_col, direction, distance = decode_action(encoded_move)
        from_pos = Point(from_row, from_col)
        to_pos = calculate_end_position(from_col, from_row, direction, distance)
        return cls(from_pos, to_pos)

    def encode(self):
        if self.from_pos.row == self.to_pos.row:  # Horizontal move
            distance = abs(self.from_pos.col - self.to_pos.col)
            direction = 2 if self.from_pos.col < self.to_pos.col else 3

        else:
            distance = abs(self.from_pos.row - self.to_pos.row)
            direction = 0 if self.from_pos.row > self.to_pos.row else 1

        return encode_action(self.from_pos.row, self.from_pos.col, direction, distance)

    def __eq__(self, other):
        return isinstance(other, Move) and self.from_pos == other.from_pos and self.to_pos == other.to_pos

    def __hash__(self):
        return hash((self.from_pos, self.to_pos))

    def __str__(self):
        return f"Move(from={self.from_pos}, to={self.to_pos})"


import numpy as np

EMPTY = 0
WHITE_PAWN = 2
BLACK_PAWN = 1
KING = 3

class Board:

    def __init__(self, board_size=11):
        assert board_size == 11, "Currently only 11x11 board is supported"
        self.size = board_size
        self.grid = np.zeros((self.size, self.size), dtype=int)
        self.set_up_board()

    def reset(self):
        self.set_up_board()

    def set_up_board(self):
        center = (self.size // 2)
        self.current_player = Player.black
        self.place_king(center)
        self.initialize_black(center)
        self.initialize_white(center)

    def place_king(self, center):
        center_point = Point(center, center)
        self.grid[center_point.row, center_point.col] = KING

    def initialize_black(self, center):
        # initialization of black for 11x11 board
        for i in range(3, self.size - 3):
            self.grid[0, i] = BLACK_PAWN
            self.grid[self.size - 1, i] = BLACK_PAWN
            self.grid[i, 0] = BLACK_PAWN
            self.grid[i, self.size - 1] = BLACK_PAWN
        self.grid[1, center] = BLACK_PAWN
        self.grid[self.size - 2, center] = BLACK_PAWN
        self.grid[center, 1] = BLACK_PAWN
        self.grid[center, self.size - 2] = BLACK_PAWN

    def initialize_white(self, center):
        # initialization of white for 11x11 board
        self.grid[center, center] = KING
        self.grid[center, center - 1] = WHITE_PAWN
        self.grid[center, center + 1] = WHITE_PAWN
        self.grid[center - 1, center] = WHITE_PAWN
        self.grid[center + 1, center] = WHITE_PAWN
        self.grid[center, center - 2] = WHITE_PAWN
        self.grid[center, center + 2] = WHITE_PAWN
        self.grid[center - 2, center] = WHITE_PAWN
        self.grid[center + 2, center] = WHITE_PAWN
        for r_off, c_off in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
             self.grid[center + r_off, center + c_off] = WHITE_PAWN

    def is_on_board(self, point):
        return 0 <= point.row < self.size and 0 <= point.col < self.size

    def get_pawn_at(self, point):
        if not self.is_on_board(point):
            return None
        return self.grid[point.row, point.col]

    def move_pawn(self, move):
        piece = self.get_pawn_at(move.from_pos)
        self.grid[move.from_pos.row, move.from_pos.col] = EMPTY
        self.grid[move.to_pos.row, move.to_pos.col] = piece

    def __str__(self) -> str:
        symbols = {0: '.', 1: 'B', 2: 'W', 3: 'K'}
        rendered = []
        for row in self.grid:
            #print(' '.join(symbols[cell] for cell in row))
            rendered.append(' '.join(symbols[cell] for cell in row))
        return '\n'.join(rendered)

board = Board()
board.move_pawn(Move(Point(0,3), Point(4,3)))
print(board)

. . . . B B B B . . .
. . . . . B . . . . .
. . . . . . . . . . .
B . . . . W . . . . B
B . . B W W W . . . B
B B . W W K W W . B B
B . . . W W W . . . B
B . . . . W . . . . B
. . . . . . . . . . .
. . . . . B . . . . .
. . . B B B B B . . .


In [238]:
move = Move(Point(0, 3), Point(4, 3))
encoded_move = move.encode()
print(f"Encoded move: {encoded_move}")
new_move = Move.from_encoded(encoded_move)
print(f"Decoded move: {new_move}")


Encoded move: 133
Decoded move: Move(from=Point(row=0, col=3), to=Point(row=4, col=3))


In [241]:
from collections import deque

class GameState:
    def __init__(self, board, next_player, previous, move):
        self.board = board
        self.next_player = next_player
        self.previous = previous
        self.last_move = move
        self.winner = None

    @classmethod
    def new_game(cls, board_size=11):
        return cls(Board(board_size), Player.white, None, None)

    def apply_move(self, move):
        new_board = Board(self.board.size)
        new_board.grid = np.copy(self.board.grid)
        new_board.move_pawn(move)

        next_state = GameState(new_board, self.next_player.other, self, move)
        next_state._check_for_capture(move.to_pos, self.next_player, move)
        next_state.is_over()
        return next_state

    def is_over(self):
        if self.winner is not None:
            return True

        king_pos = self.find_king()
        if self._is_king_captured(king_pos):
            self.winner = Player.black
            return True

        if king_pos is None: # redundant check just in case
            self.winner = Player.black
            return True

        size = self.board.size
        corners = [Point(0, 0), Point(0, size - 1), Point(size - 1, 0), Point(size - 1, size - 1)]

        if king_pos in corners:
            self.winner = Player.white
            return True

        if self._is_fortress():
            self.winner = Player.black
            return True

        return False

    def get_legal_moves(self):
        legal_moves = []
        my_pawns = (WHITE_PAWN, KING) if self.next_player == Player.white else (BLACK_PAWN, KING)
        size = self.board.size
        corners = [Point(0, 0), Point(0, size - 1), Point(size - 1, 0), Point(size - 1, size - 1)]
        throne = Point(size // 2, size // 2)

        for r in range(size):
            for c in range(size):
                piece = self.board.grid[r, c]
                if piece in my_pawns:
                    from_pos = Point(r, c)
                    for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                        for distance in range(1, size):
                            to_pos = Point(r + dr * distance, c + dc * distance)
                            if not self.board.is_on_board(to_pos):
                                break
                            if self.board.get_pawn_at(to_pos) != EMPTY:
                                break
                            move = Move(from_pos, to_pos)
                            legal_moves.append(move)




    def will_capture(self, neighbor_point, player, capture_point):
        new_point = neighbor_point
        if self.is_hostile(new_point, player):
            self.capture(capture_point)


    def _check_for_capture(self, point, player, move):
        opponent_pawn = player.other.value
        # Check if the new point is adjacent to an opponent's pawn
        d = 0 # 0:up, 1:down, 2:left, 3:right
        for neighbor in point.neighbors():
            if (self.board.is_on_board(neighbor) and
                    self.board.grid[neighbor.row, neighbor.col] == opponent_pawn):

                if d == 0:  # Up
                    self.will_capture(Point(neighbor.row - 1, neighbor.col), player, neighbor)
                elif d == 1:  # Down
                    self.will_capture(Point(neighbor.row + 1, neighbor.col), player, neighbor)
                elif d == 2:  # Left
                    self.will_capture(Point(neighbor.row, neighbor.col - 1), player, neighbor)
                elif d == 3:  # Right
                    self.will_capture(Point(neighbor.row, neighbor.col + 1), player, neighbor)

            d += 1

    def capture(self, point):
        self.board.grid[point.row, point.col] = EMPTY

    def is_hostile(self, point, player):
        # Check if the point is occupied by an opponent's pawn or is a corner point or center point
        if not self.board.is_on_board(point):
            return False
        pawn = self.board.get_pawn_at(point)
        if pawn == player.other.value:
            return True
        if pawn == KING and player == Player.black:
            # King is only hostile to black pawns
            return True

        size = self.board.size
        throne = Point(size // 2, size // 2)
        corners = [Point(0, 0), Point(0, size - 1), Point(size - 1, 0), Point(size - 1, size - 1)]
        if point in corners:
            return True

        if point == throne and (self.board.get_pawn_at(throne) == EMPTY or self.board.get_pawn_at(throne) == player.other.value):
            return True

        if pawn == EMPTY:
            return False

        return None

    def _is_king_captured(self, king_pos):
        # Check if the king is captured by checking if it is surrounded by hostile pawns
        attacking_pawns = 0
        for neighbor in king_pos.neighbors():
            if not self.is_hostile(neighbor, self.next_player):
                return False
            if self.board.get_pawn_at(neighbor) == BLACK_PAWN:
                attacking_pawns += 1

        return attacking_pawns >= 3

    def find_king(self):
        # Find the position of the king on the board
        for row in range(self.board.size):
            for col in range(self.board.size):
                if self.board.grid[row, col] == KING:
                    return Point(row, col)
        return None

    def _is_fortress(self):
        # Determine if white is stuck in a fortress by doing a bfs from the corners of the boards. If none of the white pawns are in the search space, then it is a fortress
        size = self.board.size
        d = deque([Point(0, 0), Point(0, size - 1), Point(size - 1, 0), Point(size - 1, size - 1)])
        visited = set()

        while d:
            point = d.popleft()
            if point in visited or not self.board.is_on_board(point):
                continue
            visited.add(point)

            # Check if the point is occupied by a white pawn
            if self.board.get_pawn_at(point) == WHITE_PAWN:
                return False

            # Add neighbors to the queue
            for neighbor in point.neighbors():
                if neighbor not in visited and self.board.is_on_board(neighbor):
                    d.append(neighbor)
        # If we reach here, it means all available points are either empty or occupied by black pawns
        return True

    def _is_shield_wall(self):
        pass

    def _is_exit_fort(self):
        pass








In [248]:
# test GameState
game_state = GameState.new_game()
print(game_state.board)
print("\n")
new_state = game_state.apply_move(Move(Point(0, 3), Point(3, 3)))
print(new_state.board)


. . . B B B B B . . .
. . . . . B . . . . .
. . . . . . . . . . .
B . . . . W . . . . B
B . . . W W W . . . B
B B . W W K W W . B B
B . . . W W W . . . B
B . . . . W . . . . B
. . . . . . . . . . .
. . . . . B . . . . .
. . . B B B B B . . .


. . . . B B B B . . .
. . . . . B . . . . .
. . . . . . . . . . .
B . . B . W . . . . B
B . . . W W W . . . B
B B . W W K W W . B B
B . . . W W W . . . B
B . . . . W . . . . B
. . . . . . . . . . .
. . . . . B . . . . .
. . . B B B B B . . .


In [ ]:
"""
    def move_pawn(self, player, move):
        # will need to check: pawn of current player, valid move, capture logic
        # move is legal if it doesn't go off the board and doesn't hop over another pawn
        from_row, from_col, direction, distance = decode(move)
        pawn = self.get_pawn_at(Point(from_row, from_col))
        if pawn != (WHITE_PAWN if player == Player.white else BLACK_PAWN):
            raise ValueError("Invalid pawn for the current player")
        if direction == 0:  # Up
            new_row = from_row - distance
            new_col = from_col
        elif direction == 1:  # Down
            new_row = from_row + distance
            new_col = from_col
        elif direction == 2:  # Left
            new_row = from_row
            new_col = from_col - distance
        elif direction == 3:  # Right
            new_row = from_row
            new_col = from_col + distance
        else:
            raise ValueError("Invalid direction")

        new_point = Point(new_row, new_col)
        if not self.is_on_board(new_point):
            raise ValueError("Move goes off the board")
        if self.get_pawn_at(new_point) != EMPTY:
            raise ValueError("Cannot move to a point that is already occupied by another pawn")
        self.grid[from_row, from_col] = EMPTY
        self.grid[new_point.row, new_point.col] = pawn
        self._check_for_capture(new_point, player)

    def will_capture(self, neighbor_point, player, capture_point):
            new_point = neighbor_point
            if self.grid[new_point.row, new_point.col] == player.value:
                self.capture(capture_point)
            elif not self.is_on_board(new_point):
                self.capture(capture_point)

    def capture(self, point):
        self.grid[point.row, point.col] = EMPTY


    def _check_for_capture(self, new_point, player):
        opponent_pawn = player.other.value
        # Check if the new point is adjacent to an opponent's pawn
        d = 0 # 0:up, 1:down, 2:left, 3:right
        for neighbor in new_point.neighbors():
            if (self.is_on_board(neighbor) and
                    self.grid[neighbor.row, neighbor.col] == opponent_pawn):

                if d == 0:  # Up
                    self.will_capture(Point(neighbor.row - 1, neighbor.col), player, neighbor)

                elif d == 1:  # Down
                    self.will_capture(Point(neighbor.row + 1, neighbor.col), player, neighbor)

                elif d == 2:  # Left
                    self.will_capture(Point(neighbor.row, neighbor.col - 1), player, neighbor)

                elif d == 3:  # Right
                    self.will_capture(Point(neighbor.row, neighbor.col + 1), player, neighbor)

            d += 1


    def get_valid_moves(self, player):
        pass

    def is_king(self, point):
        return self.grid[point.row, point.col] == KING


def decode(move):
    from_row = move // (11*4*10)
    rem = move % (11*4*10)

    from_col = rem // (4*10)
    rem = rem % (4*10)

    direction = rem // 10
    distance = (rem % 10) + 1

    return from_row, from_col, direction, distance

def encode(from_row, from_col, direction, distance):
    if not (0 <= from_row < 11 and 0 <= from_col < 11):
        raise ValueError("Row and column must be between 0 and 10 inclusive")
    if direction not in [0, 1, 2, 3]:
        raise ValueError("Direction must be one of: 0 (up), 1 (down), 2 (left), 3 (right)")
    if not (0 <= distance <= 10):
        raise ValueError("Distance must be between 0 and 10 inclusive")

    return from_row * (11 * 4 * 10) + from_col * (4 * 10) + direction * 10 + (distance - 1)

class Move:
    def __init__(self, from_row, from_col, direction, distance):
        self.row = from_row
        self.col = from_col
        self.direction = direction
        self.distance = distance

    @classmethod
    def from_encoded(cls, encoded_move):
        from_row, from_col, direction, distance = decode(encoded_move)
        return cls(from_row, from_col, direction, distance)

    def end_position(self):
        if self.direction == 0:  # Up
            return Point(self.row - self.distance, self.col)
        elif self.direction == 1:  # Down
            return Point(self.row + self.distance, self.col)
        elif self.direction == 2:  # Left
            return Point(self.row, self.col - self.distance)
        elif self.direction == 3:  # Right
            return Point(self.row, self.col + self.distance)
        else:
            raise ValueError("Invalid direction")

    def encode(self):
        return encode(self.row, self.col, self.direction, self.distance)

    def __eq__(self, other):
        return  isinstance(other, Move) and self.row == other.row and self.col == other.col and self.direction == other.direction and self.distance == other.distance

    def __hash__(self):
        return hash((self.row, self.col, self.direction, self.distance))

    def __str__(self):
        return f"Move(from=({self.row}, {self.col}), direction={self.direction}, distance={self.distance})"

class GameState:
    def __init__(self, board, next_player, previous, move):
        self.board = board
        self.next_player = next_player
        self.previous = previous
        self.last_move = move
        self.winner = None

    @classmethod
    def new_game(cls, board_size=11):
        return cls(Board(board_size), Player.white, None, None)

    def apply_move(self, move):
        new_board = Board(self.board.size)
        new_board.grid = np.copy(self.board.grid)
        new_board.move_pawn(move)

        next_state = GameState(new_board, self.next_player.other, self, move)
        next_state._check_for_capture(move.to_pos, self.next_player, move)


    def is_hostile(self, point, current_player):
        # Check if the point is occupied by an opponent's pawn or is a corner point or center point, king is only hostile to black pawns
        if not self.board.is_on_board(point):
            return False
        pawn = self.board.get_pawn_at(point)
        if pawn == EMPTY:
            return False
        if pawn == current_player.other.value:
            return True
        if pawn == KING and current_player == Player.white:
            # King is only hostile to black pawns
            return True
        # check if corner or center point
        if (point == Point(0, 0) or point == Point(0, self.board.size - 1) or
                point == Point(self.board.size - 1, 0) or point == Point(self.board.size - 1, self.board.size - 1) or
                point == Point(self.board.size // 2, self.board.size // 2)):
            return True

        return False

    def will_capture(self, neighbor_point, player, capture_point):
        new_point = neighbor_point
        if self.is_hostile(new_point, player):
            self.capture(capture_point)


    def _check_for_capture(self, point, player, move):
        opponent_pawn = player.other.value
        # Check if the new point is adjacent to an opponent's pawn
        d = 0 # 0:up, 1:down, 2:left, 3:right
        for neighbor in point.neighbors():
            if (self.board.is_on_board(neighbor) and
                    self.board.grid[neighbor.row, neighbor.col] == opponent_pawn):

                if d == 0:  # Up
                    self.will_capture(Point(neighbor.row - 1, neighbor.col), player, neighbor)
                elif d == 1:  # Down
                    self.will_capture(Point(neighbor.row + 1, neighbor.col), player, neighbor)
                elif d == 2:  # Left
                    self.will_capture(Point(neighbor.row, neighbor.col - 1), player, neighbor)
                elif d == 3:  # Right
                    self.will_capture(Point(neighbor.row, neighbor.col + 1), player, neighbor)

            d += 1

    def capture(self, point):
        self.board.grid[point.row, point.col] = EMPTY

    def is_hostile(self, point, current_player):
        # Check if the point is occupied by an opponent's pawn or is a corner point or center point
        if not self.board.is_on_board(point):
            return False
        pawn = self.board.get_pawn_at(point)
        if pawn == EMPTY:
            return False
        if pawn == current_player.other.value:
            return True

    @property
    def situation(self):
        return self.board, self.next_player

    def is_legal_move(self, move):
        # Check if the move is legal
        return True

    def is_over(self):
        pass

    def legal_moves(self):
        pass

    def winner(self):
        pass

"""